<a href="https://colab.research.google.com/github/Mike-Wazovsky/JetBrain-emotion-classification/blob/main/other_implementations/JetBrains_other_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **JetBrains**

In [1]:
import numpy as np
import pandas as pd
from typing import Tuple
from sklearn.utils import shuffle
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
#Delete this

In [3]:
# X, y = read_dataset('fb_sentiment.csv')
# print(X)
# print(y)

- Stanford CoreNLP parser разобраться что это
- The SVM is trained as a one-versus-all
classifier with a linear kernel (8 models are trained,
one for each emotion of EmoLex) and the TF-IDF
model (Salton and Buckley, 1988) is used for providing the input features. Input consists of a single sentence as data (transformed using the TF-IDF model)
and an array of 8 values representing the emotions as a
label. With a training/test-split of 80%/20%, the average precision-recall is about 0.93.

In [4]:
# General Pre-processing 
def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'
def preprocess(sent):
    sent = remove_users(sent)
    sent = remove_links(sent)
    sent = sent.lower() # lower case
    sent = re.sub('['+my_punctuation + ']+', ' ', sent) # strip punctuation
    sent = re.sub('\s+', ' ', sent) #remove double spacing
    sent = re.sub('([0-9]+)', '', sent) # remove numbers
    sent_token_list = [word for word in sent.split(' ')]
    sent = ' '.join(sent_token_list)
    return sent

In [5]:
# count number of characters 
def count_chars(text):
    return len(text)

# count number of words 
def count_words(text):
    return len(text.split())

# count number of capital characters
def count_capital_chars(text):
    count=0
    for i in text:
        if i.isupper():
            count+=1
    return count

# count number of capital words
def count_capital_words(text):
    return sum(map(str.isupper,text.split()))

# count number of punctuations
def count_punctuations(text):
    punctuations='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    d=dict()
    for i in punctuations:
        d[str(i)+' count']=text.count(i)
    return d

# count number of words in quotes
def count_words_in_quotes(text):
    x = re.findall("\'.\'|\".\"", text)
    count=0
    if x is None:
        return 0
    else:
        for i in x:
            t=i[1:-1]
            count+=count_words(t)
        return count
    
# count number of sentences
def count_sent(text):
    return len(nltk.sent_tokenize(text))

# calculate average word length
def avg_word_len(char_cnt,word_cnt):
    return char_cnt/word_cnt

# calculate average sentence length
def avg_sent_len(word_cnt,sent_cnt):
    return word_cnt/sent_cnt

# count number of unique words 
def count_unique_words(text):
    return len(set(text.split()))
            
# words vs unique feature
def words_vs_unique(words,unique):
    return unique/words
    
# count of hashtags
def count_htags(text):
    x = re.findall(r'(\#\w[A-Za-z0-9]*)', text)
    return len(x)

# count of mentions
def count_mentions(text):
    x = re.findall(r'(\@\w[A-Za-z0-9]*)', text)
    return len(x)

# count of stopwords
def count_stopwords(text):
    stop_words = set(stopwords.words('english'))  
    word_tokens = word_tokenize(text)
    stopwords_x = [w for w in word_tokens if w in stop_words]
    return len(stopwords_x)

# stopwords vs words
def stopwords_vs_words(stopwords_cnt,text):
    return stopwords_cnt/len(word_tokenize(text))

In [6]:
df = pd.read_csv('fb_sentiment.csv')
df['char_count'] = df["FBPost"].apply(lambda x:count_chars(x))
df['word_count'] = df["FBPost"].apply(lambda x:count_words(x))
df['sent_count'] = df["FBPost"].apply(lambda x:count_sent(x))
df['capital_char_count'] = df["FBPost"].apply(lambda x:count_capital_chars(x))
df['capital_word_count'] = df["FBPost"].apply(lambda x:count_capital_words(x))
df['quoted_word_count'] = df["FBPost"].apply(lambda x:count_words_in_quotes(x))
df['stopword_count'] = df["FBPost"].apply(lambda x:count_stopwords(x))
df['unique_word_count'] = df["FBPost"].apply(lambda x:count_unique_words(x))
df['htag_count'] = df["FBPost"].apply(lambda x:count_htags(x))
df['mention_count'] = df["FBPost"].apply(lambda x:count_mentions(x))
df['avg_wordlength']=df['char_count']/df['word_count']
df['avg_sentlength']=df['word_count']/df['sent_count']
df['unique_vs_words']=df['unique_word_count']/df['word_count']
df['stopwords_vs_words']=df['stopword_count']/df['word_count']


In [7]:
train, test = train_test_split(df, test_size=0.2)

In [8]:
train['FBPost']=train['FBPost'].apply(lambda x: preprocess(x))
test['FBPost']=test['FBPost'].apply(lambda x: preprocess(x))

In [9]:
train['FBPost'] = train['FBPost'].astype('string')
test['FBPost'] = test['FBPost'].astype('string')
train['FBPost'].fillna('', inplace = True)
test['FBPost'].fillna('', inplace = True)

In [10]:
vectorizer            =  TfidfVectorizer()
train_tf_idf_features =  vectorizer.fit_transform(train['FBPost']).toarray()
test_tf_idf_features  =  vectorizer.transform(test['FBPost']).toarray()
# Converting above list to DataFrame
train_tf_idf          = pd.DataFrame(train_tf_idf_features)
test_tf_idf           = pd.DataFrame(test_tf_idf_features)

# Saparating train and test labels from all features
train_Y               = train['Label']
test_Y                = test['Label']
# Listing all features
features = ['char_count', 'word_count', 'sent_count',
       'capital_char_count', 'capital_word_count', 'quoted_word_count',
       'stopword_count', 'unique_word_count', 'htag_count', 'mention_count',
       'avg_wordlength', 'avg_sentlength', 'unique_vs_words',
       'stopwords_vs_words']
# Finally merging all features with above TF-IDF. 
train_fch = train[features]
test_fch = test[features]


train_fch['Id'] = np.arange(0, len(train_fch))
test_fch['Id'] = np.arange(0, len(test_fch))

train_tf_idf['Id'] = np.arange(0, len(train_tf_idf))
test_tf_idf['Id'] = np.arange(0, len(test_tf_idf))

train = train_tf_idf.merge(train_fch, left_on='Id', right_on='Id', how='inner')
test = test_tf_idf.merge(test_fch, left_on='Id', right_on='Id', how='inner')

train.columns = train.columns.astype(str)
test.columns = test.columns.astype(str)

<ipython-input-10-54278f600cab>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_fch['Id'] = np.arange(0, len(train_fch))
<ipython-input-10-54278f600cab>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_fch['Id'] = np.arange(0, len(test_fch))


In [11]:
X_train, X_test, y_train, y_test = train_test_split(train, train_Y, test_size=0.2, random_state = 42)# Random Forest Classifier
RFClassifier = RandomForestClassifier(n_estimators = 1000, min_samples_split = 15, random_state = 42)
RFClassifier.fit(X_train, y_train)
y_pred = RFClassifier.predict(X_test)
val_y_pred = RFClassifier.predict(test)
print("Accuracy => ", round(accuracy_score(y_pred, y_test)*100, 2))
print("\nRandom Forest Classifier results: \n")
print(classification_report(y_test, y_pred, target_names = ['N', 'O', 'P']))
print("Validation Accuracy => ", round(accuracy_score(val_y_pred, test_Y)*100, 2))
print("\nValidation Random Forest Classifier results: \n")
print(classification_report(test_Y, val_y_pred, target_names = ['N', 'O', 'P']))

Accuracy =>  68.75

Random Forest Classifier results: 

              precision    recall  f1-score   support

           N       1.00      0.08      0.14        13
           O       0.65      0.25      0.36        44
           P       0.69      0.95      0.80       103

    accuracy                           0.69       160
   macro avg       0.78      0.43      0.43       160
weighted avg       0.70      0.69      0.63       160

Validation Accuracy =>  67.0

Validation Random Forest Classifier results: 

              precision    recall  f1-score   support

           N       0.00      0.00      0.00        13
           O       0.59      0.17      0.27        58
           P       0.68      0.96      0.79       129

    accuracy                           0.67       200
   macro avg       0.42      0.38      0.35       200
weighted avg       0.61      0.67      0.59       200



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
